In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/

/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs


/Users/karolinegriesbach/.local/share/virtualenvs/consumption-based-costs-wGvc2ut4/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [63]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

from src.utils.databricks_pp import handle_databricks_cost
from src.utils.azure_pp import handle_azure_costs
from src.utils.aws_pp import handle_aws_costs

In [10]:
from_date = "2024-01-01"
to_date="2025-06-12"
timestamp = "2025-06-13 11:40:40.049356"
path_to_dir = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/{timestamp}/"
path_to_data=f"{path_to_dir}final_costs_with_azure_and_aws_and_db_{from_date}_{to_date}.csv"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/infrastructure_insights/"

In [11]:
os.makedirs(path_to_save, exist_ok=True)

# Load Data

In [ ]:
df = pd.read_csv(path_to_data)
df = df[[col for col in df.columns if "Unnamed" not in col]]


/var/folders/yg/6h14_gl52p7_rhsrqbdjv0br0000gn/T/ipykernel_22768/2110916664.py:1: DtypeWarning: Columns (4,5,6,7,8,9,11,12,13,14,15,17,18,19,20,22,23,24,27,28,31,38,39,40,41,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_data)


# PP Data

In [20]:
df = df[df["cloud"].isnull() == False]

In [21]:
df = handle_databricks_cost(df)
df = handle_aws_costs(df)
df = handle_azure_costs(df)
df

/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/src/utils/databricks_pp.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["serviceName"] = np.where(
/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/src/utils/databricks_pp.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["serviceCategoryName"] = np.where(
/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/src/utils/databricks_pp.py:18: SettingWithCopyWarning: 
A value is trying t

,Deployments,tenant,deployment_node_name,audience,schedule,use_conversion_table_targeting,max_model_age_in_days,node_retraining_reset,Prefect_Deployments,timestamp,...,count_services,charge_by_service,effective_price,billing_currency,resource_id,resource_name,resource_group,amortized_cost,cluster_id,servicename_aws
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,retraining,asambeauty,16cpu256gib,6687daf21767991e3bdfb414,16 2 * * *,NaN,14.0,30cpu476gib,retraining,2024-10-16 00:36:36.760786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN,None
94573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN,None
94574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN,None
94575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN,None


# Get unique serviceName per account - grouped by 

In [43]:
min_date = df["date"].min()
max_date = pd.to_datetime(df["date"], format="mixed", utc=True).dt.strftime("%Y-%m-%d").max()
print(min_date)
print(max_date)

2024-05-05
2025-06-12


In [30]:
cloud_services = df.groupby(by=["cloud","serviceCategoryName","serviceName"])["date"].count().reset_index().sort_values(by=["cloud","serviceCategoryName","serviceName"],ascending=True)
cloud_services

,cloud,serviceCategoryName,serviceName,date
0,aws,APN1-FreeEventsRecorded,APN1-FreeEventsRecorded,1
1,aws,CA-DataTransfer-Out-Bytes,CA-DataTransfer-Out-Bytes,1
2,aws,CA-Requests-Tier2-HTTPS,CA-Requests-Tier2-HTTPS,1
3,aws,ComputeSP:3yrNoUpfront,ComputeSP:3yrNoUpfront,404
4,aws,DashboardsUsageHour-Basic,DashboardsUsageHour-Basic,404
...,...,...,...,...
407,stackit,Storage,Block Storage for PostgreSQL Premium-Performan...,45
408,stackit,Storage,Block Storage for disk volumes Premium-Capacit...,247
409,stackit,Storage,Block Storage for disk volumes Premium-Perform...,1
410,stackit,Storage,Block Storage for disk volumes Premium-Perform...,178


In [45]:
cloud_services.to_csv(f"{path_to_save}unique_cloud_services_{min_date}_{max_date}_{datetime.now()}.csv")

In [48]:
cloud_services.groupby(by="cloud")["serviceCategoryName"].nunique()

cloud
aws           278
azure          11
databricks      3
stackit         6
Name: serviceCategoryName, dtype: int64

# Load labeled list via ChatGPT

In [61]:
df_labeled = pd.read_csv(f"{path_to_save}Cost_Data_with_Detailed_Documentation_Links_v2.csv",sep=";", index_col=0)

In [62]:
df_labeled

,cloud,serviceCategoryName,serviceName,date,label,Group,Open Questions,Description,Documentation Link,serviceName_without_country_string
Unnamed: 0,,,,,,,,,,
3,aws,ComputeSP:3yrNoUpfront,ComputeSP:3yrNoUpfront,404,NaN,Servers,NaN,NaN,NaN,NaN
264,aws,Invalidations,Invalidations,13,NaN,Other,NaN,NaN,NaN,NaN
10,aws,EUC1-AFS1-AWS-In-Bytes,EUC1-AFS1-AWS-In-Bytes,54,NaN,Data Transfer,Cape Town?,NaN,This usage type appears when traffic flows via...,AFS1-AWS-In-Bytes
11,aws,EUC1-AFS1-AWS-Out-Bytes,EUC1-AFS1-AWS-Out-Bytes,53,NaN,Data Transfer,Cape Town?,NaN,This usage type appears when traffic flows via...,AFS1-AWS-Out-Bytes
12,aws,EUC1-AKL1-AWS-In-Bytes,EUC1-AKL1-AWS-In-Bytes,47,NaN,Data Transfer,Neuseeland?,NaN,NaN,AKL1-AWS-In-Bytes
...,...,...,...,...,...,...,...,...,...,...
388,stackit,Compute Engine,General Purpose Server-g1.3-EU01,922,NaN,Servers,NaN,NaN,NaN,g1.3-EU01
389,stackit,Compute Engine,General Purpose Server-g1.4-EU01,156,NaN,Servers,NaN,NaN,NaN,g1.4-EU01
395,stackit,Compute Engine,Memory Optimized Server-m1.3-EU01,1093,NaN,Servers,NaN,NaN,NaN,m1.3-EU01


In [64]:
df_labeled["serviceName_without_country_string"] = df_labeled["serviceName"].str.split("-").str[1::].str.join("-")
df_labeled["serviceName_without_country_string"] = np.where(
    df_labeled["serviceName_without_country_string"].str.contains("AWS-Out-Bytes"),
    "AWS-Out-Bytes",
    df_labeled["serviceName_without_country_string"],
)
df_labeled["serviceName_without_country_string"] = np.where(
    df_labeled["serviceName_without_country_string"].str.contains("AWS-In-Bytes"),
    "AWS-In-Bytes",
    df_labeled["serviceName_without_country_string"],
)
df_labeled["serviceName_without_country_string"].value_counts()

serviceName_without_country_string
AWS-Out-Bytes              68
AWS-In-Bytes               66
                            8
FreeEventsRecorded          6
DataTransfer-Out-Bytes      4
                           ..
PublicIPv4:IdleAddress      1
PublicIPv4:InUseAddress     1
Request                     1
Request-ARM                 1
t1.2-EU01-m                 1
Name: count, Length: 250, dtype: int64

In [65]:
df_labeled.sort_values(by=["cloud","serviceName_without_country_string"]).to_csv(f"{path_to_save}Cost_Data_with_Detailed_Documentation_Links_without_contry_strings.csv",sep=";")